In [24]:
import clickhouse_connect
import pandas as pd
import numpy as np

In [25]:
client = clickhouse_connect.get_client(
    host='clickhouse-0-0.umetea.net',
    port=443,
    username='ml_ume',
    password='hDAoDvg8x552bH',
    database='dw',
    verify=False
)

customers_order = client.query_df("""
SELECT
    customer_id,
    count(DISTINCT order_id) AS order_cnt_12m,
    sum(total_amount) AS order_amt_12m,
    countIf(order_id, created_at_pt >= addMonths(now(), -6)) AS order_cnt_6m,
    sumIf(total_amount, created_at_pt >= addMonths(now(), -6)) AS order_amt_6m,
    min(created_at_pt) AS first_order_date,
    max(created_at_pt) AS last_order_date
FROM fact_orders
WHERE created_at_pt >= addMonths(now(), -12)
GROUP BY customer_id
""")

customers_lottery = client.query_df("""
SELECT
    customer_id,
    max(CASE WHEN joined=1 THEN 1 ELSE 0 END) AS lottery_joined,
    max(CASE WHEN redeemed_time_pt IS NOT NULL THEN 1 ELSE 0 END) AS lottery_redeemed
FROM dw.fact_lottery_histories
WHERE customer_created_at_pt >= addMonths(now(), -12)
GROUP BY customer_id
""")

promotion_past_12m = client.query_df("""
SELECT
    customer_id,
    count(DISTINCT order_id) AS coupon_order_cnt_12m
FROM dw.fact_orders
WHERE created_at_pt >= addMonths(now(), -12)
  AND (
      (campaign_names IS NOT NULL AND length(campaign_names) > 0)
      OR (source_type LIKE '%coupon%' OR source_type LIKE '%促销%')
      OR (discount > 0)
  )
GROUP BY customer_id

""")

category_preference = client.query_df("""
SELECT
    customer_id,
    arrayJoin([category_name]) AS major_category, -- 这里可进一步聚合求最多的品类
    count(*) AS category_order_cnt
FROM dw.fact_order_item_variations
WHERE created_at_pt >= addMonths(now(), -12)
GROUP BY customer_id, category_name
ORDER BY customer_id, category_order_cnt DESC
""")

client.close()


In [31]:
customers_lottery

,customer_id,lottery_joined,lottery_redeemed
0,1C91QQ3WB4AR8QX1F79W97MCVG,1,1
1,MF85TA6ADZJE5RMV0N9K3QWR70,1,1
2,2MDJ8E2N0C02BXZBRDWFD34W50,1,1
3,B8DCSWTSNS31K9PPXAPBGRJ2PC,1,1
4,FM1XWBW5957TD32X163ZN3Q5MG,1,1
...,...,...,...
2006,XVK5E2G88XB7A8V8967AX2J04M,1,1
2007,D9BKC698YX9YV83C2XQGE0KN1M,1,1
2008,67M27WPC2B69T49NFN4J4ZZV2C,1,1
2009,2HBHS3SPWF3AY5TDY6WWW75F9R,1,1


In [33]:
category_preference

,customer_id,major_category,category_order_cnt
0,0001VEK5ZX181768HPGV8M8V0G,Fruit Tea,2
1,0001VEK5ZX181768HPGV8M8V0G,Matcha,1
2,0001VEK5ZX181768HPGV8M8V0G,Caffeine-Free Drinks,1
3,0001VEK5ZX181768HPGV8M8V0G,Slushies,1
4,0001VEK5ZX181768HPGV8M8V0G,Milk Tea,1
...,...,...,...
673157,<NA>,Coconut,155
673158,<NA>,Milkshake,109
673159,<NA>,Diy Gelato,11
673160,<NA>,Fresh Tea(Lo),9


In [27]:
"""
AI增长引擎Demo - 基于FBR实际数据
"""
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from prophet import Prophet
import xgboost as xgb
from sklearn.model_selection import train_test_split
import streamlit as st
import plotly.graph_objects as go
import plotly.express as px
import clickhouse_connect

# ClickHouse连接配置
CLICKHOUSE_CONFIG = {
    'host': 'clickhouse-0-0.umetea.net',
    'port': 443,
    'database': 'dw',
    'user': 'ml_ume',
    'password': 'hDAoDvg8x552bH',
    'verify':False
}


class FBRGrowthEngine:
    """FBR餐饮AI增长引擎"""

    def __init__(self):
        self.ch_client = clickhouse_connect.get_client(**CLICKHOUSE_CONFIG)

    def load_sales_data(self, days_back=90):
        """加载历史销售数据"""
        query = f"""
        SELECT
            toDate(created_at_pt) as date,
            location_id,
            location_name,
            COUNT(DISTINCT order_id) as order_count,
            SUM(item_qty) as total_items,
            SUM(item_total_amt) as total_revenue,
            SUM(item_discount) as total_discount,
            AVG(item_total_amt) as avg_order_value,
            COUNT(DISTINCT customer_id) as unique_customers,
            SUM(CASE WHEN has(is_new_users, 1) THEN 1 ELSE 0 END) as new_customers,
            COUNT(DISTINCT CASE WHEN is_loyalty = 1 THEN customer_id END) as loyalty_customers
        FROM dw.fact_order_item_variations
        WHERE created_at_pt >= today() - {days_back}
            AND pay_status = 'COMPLETED'
        GROUP BY date, location_id, location_name
        ORDER BY date DESC
        """
        return self.ch_client.query_df(query)

    def load_item_performance(self, days=30):
        """加载商品销售表现"""
        query = f"""
        SELECT
            item_name,
            category_name,
            COUNT(DISTINCT order_id) as order_count,
            SUM(item_qty) as units_sold,
            SUM(item_total_amt) as revenue,
            AVG(item_amt) as avg_price,
            SUM(item_discount) as total_discount,
            COUNT(DISTINCT customer_id) as unique_buyers,
            toDate(created_at_pt) as date
        FROM dw.fact_order_item_variations
        WHERE created_at_pt >= today() - {days}
            AND pay_status = 'COMPLETED'
            AND item_name IS NOT NULL
        GROUP BY item_name, category_name, date
        ORDER BY revenue DESC
        """
        return self.ch_client.query_df(query)

    def load_customer_segments(self):
        """加载客户分群数据"""
        query = """
        SELECT
            COUNT(DISTINCT customer_id) as customer_count,
            SUM(order_final_total_amt) as total_revenue,
            AVG(order_final_avg_amt) as avg_order_value,
            CASE
                WHEN high_value_customer = 1 THEN '高价值客户'
                WHEN key_development_customer = 1 THEN '重点发展客户'
                WHEN regular_customer = 1 THEN '普通客户'
                WHEN critical_win_back_customer = 1 THEN '重点挽回客户'
                ELSE '其他'
            END as customer_segment
        FROM ads.customer_profile
        WHERE order_final_total_cnt > 0
        GROUP BY customer_segment
        """
        return self.ch_client.query_df(query)

    def predict_next_week_sales(self, location_id=None):
        """预测下周销量 - 使用Prophet"""
        # 加载历史数据
        sales_data = self.load_sales_data(days_back=180)

        if location_id:
            sales_data = sales_data[sales_data['location_id'] == location_id]

        # 准备Prophet数据格式
        prophet_data = sales_data.groupby('date').agg({
            'total_revenue': 'sum',
            'order_count': 'sum'
        }).reset_index()
        prophet_data.columns = ['ds', 'y', 'order_count']

        # 训练模型
        model = Prophet(
            yearly_seasonality=True,
            weekly_seasonality=True,
            daily_seasonality=False,
            seasonality_mode='multiplicative'
        )

        # 添加额外特征
        model.add_regressor('order_count')

        model.fit(prophet_data)

        # 预测未来7天
        future = model.make_future_dataframe(periods=7)
        future['order_count'] = prophet_data['order_count'].mean()  # 使用平均值

        forecast = model.predict(future)

        return forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(7)

    def analyze_sales_drivers(self):
        """分析销售驱动因素"""
        # 加载最近30天数据
        recent_data = self.load_sales_data(days_back=30)

        # 创建特征
        recent_data['day_of_week'] = pd.to_datetime(recent_data['date']).dt.dayofweek
        recent_data['is_weekend'] = recent_data['day_of_week'].isin([5, 6]).astype(int)
        recent_data['discount_rate'] = recent_data['total_discount'] / (
                    recent_data['total_revenue'] + recent_data['total_discount'])
        recent_data['new_customer_rate'] = recent_data['new_customers'] / recent_data['unique_customers']
        recent_data['loyalty_rate'] = recent_data['loyalty_customers'] / recent_data['unique_customers']

        # XGBoost特征重要性分析
        features = ['day_of_week', 'is_weekend', 'discount_rate', 'new_customer_rate', 'loyalty_rate',
                    'avg_order_value']
        X = recent_data[features]
        y = recent_data['total_revenue']

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        model = xgb.XGBRegressor(n_estimators=100, max_depth=5, random_state=42)
        model.fit(X_train, y_train)

        # 获取特征重要性
        importance = pd.DataFrame({
            'feature': features,
            'importance': model.feature_importances_
        }).sort_values('importance', ascending=False)

        return importance, model

    def generate_recommendations(self):
        """生成智能建议"""
        # 分析各维度数据
        item_perf = self.load_item_performance()
        customer_segments = self.load_customer_segments()
        sales_forecast = self.predict_next_week_sales()

        recommendations = []

        # 1. 爆品推荐
        top_items = item_perf.nlargest(5, 'revenue')
        slow_items = item_perf[item_perf['units_sold'] < item_perf['units_sold'].quantile(0.2)]

        if len(slow_items) > 0:
            recommendations.append({
                'type': '库存优化',
                'priority': '高',
                'action': f"以下商品销售缓慢，建议促销清仓：{', '.join(slow_items['item_name'].head(3))}",
                'expected_impact': '减少库存积压，提升资金周转率15%'
            })

        # 2. 客户营销建议
        if 'critical_win_back_customer' in customer_segments['customer_segment'].values:
            win_back_count = \
            customer_segments[customer_segments['customer_segment'] == '重点挽回客户']['customer_count'].iloc[0]
            recommendations.append({
                'type': '客户挽回',
                'priority': '高',
                'action': f"发现{win_back_count}位重点挽回客户，建议发送专属优惠券",
                'expected_impact': '预计挽回30%流失客户，增加月收入8%'
            })

        # 3. 销售预测建议
        next_week_avg = sales_forecast['yhat'].mean()
        current_week_avg = self.load_sales_data(days_back=7)['total_revenue'].mean()

        if next_week_avg < current_week_avg * 0.9:
            recommendations.append({
                'type': '销售预警',
                'priority': '高',
                'action': '预测下周销量可能下滑，建议提前准备促销活动',
                'expected_impact': '及时干预可避免10%的销售损失'
            })

        return recommendations

In [28]:
engine = FBRGrowthEngine()

In [29]:
today_data = engine.load_sales_data(days_back=1)

In [30]:
today_data

,date,location_id,location_name,order_count,total_items,total_revenue,total_discount,avg_order_value,unique_customers,new_customers,loyalty_customers
0,2025-07-28,LWT1BQ7SHV954,UMe-CA-Sacramento,98,218,1673.13,72.92,8.450152,62,42,29
1,2025-07-28,LVTJHAGAMD2ZY,UMe-IL-Evanston,90,148,1043.23,112.59,7.903258,69,63,27
2,2025-07-28,LNEY4KP8QJ452,UMe-CA-Livermore,113,183,1300.40,15.15,7.388636,99,139,15
3,2025-07-28,L9AZSM88ZZNZ6,UMe-CA-Millbrae,83,175,1290.14,63.37,8.323484,58,28,40
4,2025-07-28,L3T37TFC0S7WB,UMe-CA-Santa Cruz,111,177,1261.16,19.75,7.784938,81,46,17
5,2025-07-28,LMF86PJ371T9T,UMe-CA-Cupertino,158,307,2384.18,59.74,8.545448,93,62,53
6,2025-07-28,LJKR8FGH3P6MQ,UMe-CA-San Ramon,77,147,1082.07,28.90,8.075149,54,46,18
7,2025-07-28,LE8TXAK2DHED2,UMe-CA-fremont-Pacific Commons,131,260,1931.09,65.59,8.113824,106,94,40
8,2025-07-28,LM718ND6CZG94,UMe-IL-Downtown,57,81,598.08,14.20,8.192877,49,59,5
9,2025-07-28,LAX1ETYJAWM7A,UMe-CA-Davis,106,212,1551.88,54.51,8.082708,70,39,51


In [33]:
yesterday_data = engine.load_sales_data(days_back=2)#.iloc[1:2]